In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib
import os
import pyproj
import shapely
from scipy import interpolate
import osmnx as ox
from mpetools import get_info_islands, timeseries_environmental_remotesensing_CoastSat, pre_timeseries_steps, add_ecological_coastal_units, timeseries_environmental_remotesensing_area
import ee
import geopandas as gpd
import geemap

try:
    ee.Initialize()
except:
    ee.Authenticate()
    ee.Initialize()

%matplotlib qt

In [2]:
island = 'Dhiyadhoo'
country = 'Maldives'
#island_info = get_info_islands.retrieve_info_island(island, country)
#island_info = timeseries_environmental_remotesensing_CoastSat.TimeSeriesCoastSat(island, country, date_range=['2000-01-01', '2022-12-31'], overwrite=True, reference_shoreline_transects_only=True).main()
#island_info = add_ecological_coastal_units.AddEcologicalCoastalUnits(island, country, overwrite=True).main()
island_info = timeseries_environmental_remotesensing_area.TimeSeriesVegetation(island, country, overwrite=True).main()


-------------------------------------------------------------------
RETRIEVING ISLAND INFORMATION
Island: Dhiyadhoo, Maldives
-------------------------------------------------------------------

~ The following information is available: ~

general_info
               island
               country
               part of
               located in the administrative territorial entity
               located in or next to body of water
spatial_reference
               latitude
               longitude
               polygon
               polygon_OSM
               transects_ECU_characteristics
               reference_shoreline
               transects
image_collection_dict
               description
               S2
               L7
               L8
               L9
timeseries_vegetation
               description
               description_timeseries
               source
               mask_total_vegetation_S2
               mask_coastal_vegetation_S2
               mask_transects

In [19]:
polygon_gee = island_info['spatial_reference']['polygon']
imgcoll = island_info['image_collection_dict']['L8'].first().clip(polygon_gee)

# Get the projection and scale of the existing image
projection = imgcoll.select('SR_B1').projection()
scale = imgcoll.select('SR_B1').projection().nominalScale()

maskk = ee.Array([np.asarray(island_info['timeseries_vegetation']['mask_total_vegetation_L8'], dtype=float).tolist()])
mask_img = ee.Image(maskk).reproject(projection, scale=scale)
mask_img.getInfo()
img_nb = imgcoll.addBands(mask_img)


In [31]:
arrimg = imgcoll.select('SR_B1').toArray().toArray(1).arrayProject([0])
comp = ee.Image(maskk).matrixMultiply(arrimg)
comp.getInfo()

EEException: Image.matrixMultiply: 2D array inputs only, got 3 and 1.

In [32]:
img_nb

In [28]:
nimg = img_nb.select('SR_B1')

In [19]:
iimg = imgcoll.select('SR_B1').multiply(mask_img)
iimg.getInfo()

{'type': 'Image',
 'bands': [{'id': 'SR_B1',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 16711425,
    'dimensions': 2},
   'dimensions': [49, 56],
   'origin': [4983, 7407],
   'crs': 'EPSG:32643',
   'crs_transform': [30, 0, 188985, 0, -30, 275715]}]}

In [6]:
# Load an example image (replace this with your own image)
existing_image = imgcoll

# Create a NumPy binary mask (example data)
numpy_mask_array = np.random.randint(0, 2, (existing_image.getInfo()["bands"][0]["dimensions"][0], existing_image.getInfo()["bands"][0]["dimensions"][1]))

# Get the number of bands in the existing image
num_bands = existing_image.bandNames().length().getInfo()

# Create a list of images for each band of the mask
mask_images = [ee.Image(numpy_mask_array.tolist()).rename([f"mask_{i+1}"]) for i in range(num_bands)]

# Stack the list of mask images into a single image
mask_image = ee.ImageCollection(mask_images).toBands()

# Apply the mask to the existing image
updated_image = existing_image.updateMask(mask_image)


In [ ]:
polygon_gee = island_info['spatial_reference']['polygon']

m = geemap.Map(center=[island_info['spatial_reference']['latitude'], island_info['spatial_reference']['longitude']], zoom=11)
m.addLayer(img_nb.select('constant').clip(polygon_gee), vis_params={'min':0, 'max': 1})
m

In [62]:
ref_shoreline = island_info['spatial_reference']['reference_shoreline']
transects = island_info['spatial_reference']['transects']
extent = np.array(island_info['spatial_reference']['polygon'].getInfo()['coordinates'][0])

In [63]:
import pyproj

# Define the source and target coordinate systems
src_crs = pyproj.CRS('EPSG:4326')
tgt_crs = pyproj.CRS('EPSG:3857')

# Create a transformer
transformer = pyproj.Transformer.from_crs(src_crs, tgt_crs, always_xy=True)

# Reproject the data
x_reprojected, y_reprojected = transformer.transform(extent[:, 0], extent[:, 1])

# Combine data into a numpy array  
extent = np.column_stack((x_reprojected, y_reprojected))

In [64]:
plt.plot(extent[:, 0], extent[:, 1], 'k', label='Extent')
plt.plot(ref_shoreline[:, 0], ref_shoreline[:, 1], 'k', label='Reference shoreline')
for i in transects.keys():
    plt.plot(transects[i][:, 0], transects[i][:, 1], label='Transect {}'.format(i))
plt.axis('equal')
plt.show()

In [35]:
import shapely

pol = shapely.geometry.Polygon(ref_shoreline)
ls = shapely.geometry.LineString(ref_shoreline)
ls_t = shapely.geometry.LineString(transects[1])

In [37]:
pol = shapely.geometry.Polygon(ref_shoreline)
ls = shapely.geometry.LineString(ref_shoreline)
ls_t = shapely.geometry.LineString(transects[1])

# Calculate the bounding box of the polygon
min_x, min_y = np.min(extent, axis=0)
max_x, max_y = np.max(extent, axis=0)

# Define the grid parameters
grid_resolution = 20  # Adjust this to control the grid resolution
x_grid = np.arange(min_x, max_x, grid_resolution)
y_grid = np.arange(min_y, max_y, grid_resolution)

# Create a meshgrid from the x and y grid
xx, yy = np.meshgrid(x_grid, y_grid)

# Create an empty mask array with the same shape as the meshgrid
mask = np.zeros(xx.shape, dtype=bool)

# Check if each point in the meshgrid is inside the polygon
for i in range(xx.shape[0]):
    for j in range(xx.shape[1]):
        point = shapely.geometry.Point(xx[i, j], yy[i, j])
        if pol.contains(point) and point.distance(ls) < 100 and point.distance(ls_t) < 100:
            mask[i, j] = True

# Display the meshgrid and the polygon
plt.figure(figsize=(8, 8))
plt.imshow(mask, extent=[min_x, max_x, min_y, max_y], origin='lower', cmap='binary')
plt.xlabel('X')
plt.ylabel('Y')
plt.title('Meshgrid Inside Polygon')
plt.show()

In [ ]:
# create binary image of the reference shoreline (1 where the shoreline is 0 otherwise)
im_binary = np.zeros(im_shape)
for j in range(len(ref_sl_pix_rounded)):
    im_binary[ref_sl_pix_rounded[j,1], ref_sl_pix_rounded[j,0]] = 1
im_binary = im_binary.astype(bool)

# dilate the binary image to create a buffer around the reference shoreline
max_dist_ref_pixels = np.ceil(settings['max_dist_ref']/pixel_size)
se = morphology.disk(max_dist_ref_pixels)
im_buffer = morphology.binary_dilation(im_binary, se)

In [132]:
transect_characterisation = {}

for key in transects.keys():
    transect = transects[key]
    linestring_transect = shapely.geometry.LineString(transect)
    try:
        ddf = gdf[gdf.index==np.argwhere(np.array([linestring_transect.intersects(geomm) for geomm in geom])).flatten()[0]].set_index('MasterKey')
    except:
        continue

    dict_rename = {'MEAN_SIG_W': 'Mean Significant Wave Height', 
                'TIDAL_RANG': 'Tidal Range', 
                'CHLOROPHYL': 'Chlorophyll-a', 
                'TURBIDITY': 'Turbidity',
                'TEMP_MOIST': 'Temperature Moist',
                'EMU_PHYSIC': 'EMU Physical',
                'REGIONAL_S': 'Regional Sinuosity',
                'MAX_SLOPE': 'Max Slope',
                'OUTFLOW_DE': 'Outflow Density',
                'ERODIBILIT': 'Erodibility',
                'LENGTH_GEO': 'Geographical Length',
                'chl_label': 'Chlorophyll-a Descriptor',
                'river_labe': 'River Descriptor',
                'sinuosity_': 'Sinuosity Descriptor',
                'slope_labe': 'Slope Descriptor',
                'tidal_labe': 'Tidal Descriptor',
                'turbid_lab': 'Turbidity Descriptor',
                'wave_label': 'Wave Descriptor',
                'CSU_Descri': 'CSU Descriptor',
                'CSU_ID': 'CSU ID',
                'OUTFLOW__1': 'Outflow Density Rescaled',
                'Shape_Leng': 'Shape Length',
                'geometry': 'Geometry'}

    ddf = ddf.rename(columns=dict_rename)
    transect_characterisation[key] = ddf


In [6]:
def get_reference_shoreline(island, country):
    
    # Check if OSM data is available
    #try:
    gdf_coastline = ox.geometries_from_place(', '.join([island, country]), {'natural': 'coastline'})
    print('OSM coastline data available for this island.')

    #except:
        #print('No OSM data available for this island, we will manually define the reference shoreline.')
        #reference_shoreline = SDS_preprocess.get_reference_sl(metadata, settings)
    
        #return reference_shoreline

    gdf_coastline_geometry = np.array(gdf_coastline['geometry'])

    for idx_coastline in range(len(gdf_coastline_geometry)):

        if type(gdf_coastline_geometry[idx_coastline]) == shapely.geometry.linestring.LineString:
            x = np.array([gdf_coastline_geometry[idx_coastline].xy[0][i] for i in range(len(gdf_coastline_geometry[idx_coastline].xy[0]))])
            y = np.array([gdf_coastline_geometry[idx_coastline].xy[1][i] for i in range(len(gdf_coastline_geometry[idx_coastline].xy[1]))])
        
        elif type(gdf_coastline_geometry[idx_coastline]) == shapely.geometry.polygon.Polygon:
            x = np.array([gdf_coastline_geometry[idx_coastline].exterior.xy[0][i] for i in range(len(gdf_coastline_geometry[idx_coastline].exterior.xy[0]))])
            y = np.array([gdf_coastline_geometry[idx_coastline].exterior.xy[1][i] for i in range(len(gdf_coastline_geometry[idx_coastline].exterior.xy[1]))])                

        for pts in range(len(x)-1):
            xx = np.array([x[pts], x[pts+1]])
            yy = np.array([y[pts], y[pts+1]])
            x_new = np.linspace(xx[0], xx[1], 300)
            f = interpolate.interp1d(xx, yy)
            if pts == 0:
                xx_i = x_new
                yy_i = f(x_new)
            else:
                xx_i = np.concatenate((xx_i, x_new))
                yy_i = np.concatenate((yy_i, f(x_new)))

        if idx_coastline == 0:
            x_all = xx_i
            y_all = yy_i

        else:
            x_all = np.concatenate((x_all, xx_i))
            y_all = np.concatenate((y_all, yy_i))
            
    # Define the source and target coordinate systems
    src_crs = pyproj.CRS('EPSG:4326')
    tgt_crs = pyproj.CRS('EPSG:3857')

    # Create a transformer
    transformer = pyproj.Transformer.from_crs(src_crs, tgt_crs, always_xy=True)

    # Reproject the data
    x_reprojected, y_reprojected = transformer.transform(x_all, y_all)

    # Combine data into a numpy array  
    reference_shoreline = np.column_stack((x_reprojected, y_reprojected))

    return reference_shoreline

In [24]:
island = 'Kondey'
country = 'Maldives'

In [25]:
ref_shoreline = get_reference_shoreline(island, country)

OSM coastline data available for this island.


C:\Users\myriampe\AppData\Local\Temp\ipykernel_8892\3084878712.py:5: UserWarning: The `geometries` module and `geometries_from_X` functions have been renamed the `features` module and `features_from_X` functions. Use these instead. The `geometries` module and function names are deprecated and will be removed in a future release.
  gdf_coastline = ox.geometries_from_place(', '.join([island, country]), {'natural': 'coastline'})


In [12]:
plt.plot(ref_shoreline[:, 0], ref_shoreline[:, 1], 'k', label='Reference shoreline')

In [26]:
desired_distance = 100

In [27]:
list_idx = []
s = 0
while s < len(ref_shoreline)-1:
    print(s)
    list_idx.append(s)
    i = s + 1
    dist = np.hypot((ref_shoreline[s, 0]-ref_shoreline[i, 0]), (ref_shoreline[s, 1]-ref_shoreline[i, 1]))
    while dist < desired_distance:
        i += 1
        dist = np.hypot((ref_shoreline[s, 0]-ref_shoreline[i, 0]), (ref_shoreline[s, 1]-ref_shoreline[i, 1]))
        if i >= len(ref_shoreline)-1:
            break
    s = i


0
245
594
807
1011
1206
1391
1607
1961
2248
2486
2789
2984
3442
3908
4163
4601
4774
5132
5393
5704
5756
5808
5860
5912
5964
6282
6462
6676
7034
7435
7718
7908
8069
8541
8885
9182
9425
9657
10064
10351
10702
11152
11347
11787
12328
13030
13588
13857
13938
14019


In [28]:
list_idx = np.array(list_idx)

In [29]:
double_coords = np.row_stack((np.flip(ref_shoreline, axis=0), ref_shoreline))

In [30]:
polygon_shoreline = shapely.geometry.Polygon(ref_shoreline)

In [31]:
dict_transects = {}
plt.plot(ref_shoreline[:, 0], ref_shoreline[:, 1], color='k', lw=2)

#for idx_transect, idx in enumerate(np.arange(int(np.shape(double_coords)[0]/2), np.shape(double_coords)[0], int(np.shape(ref_shoreline)[0] / 100))):
for idx_transect, idx in enumerate(int(len(double_coords)/2) + list_idx):
    x = double_coords[(idx-2):(idx+4), 0]
    y = double_coords[(idx-2):(idx+4), 1]

    plt.plot(x, y, marker='o', ls=' ')

    m, b = np.polyfit(x, y, 1)
    plt.plot(x, m*x + b)

    middle_point_idx = int(len(x)/2 - 0.5)
    m_perp = -1/m
    b_perp = y[middle_point_idx] - m_perp*x[middle_point_idx]

    x_long = np.linspace(np.min(double_coords[:, 0]) - 0.5*(np.max(double_coords[:, 0]) - np.min(double_coords[:, 0])), np.max(double_coords[:, 0]) + 0.5*(np.max(double_coords[:, 0]) - np.min(double_coords[:, 0])), 5000)
    y_long = m_perp*x_long + b_perp

    transect_points = np.array([shapely.geometry.Point(x, y) for (x, y) in zip(x_long, y_long)])
    idx_inside_polygon = np.argwhere(polygon_shoreline.contains(transect_points)).flatten()

    p1, p2 = idx_inside_polygon[0], idx_inside_polygon[-1]
    dist1 = np.hypot(x_long[p1] - x[middle_point_idx], y_long[p1] - y[middle_point_idx])
    dist2 = np.hypot(x_long[p2] - x[middle_point_idx], y_long[p2] - y[middle_point_idx])

    maxx = np.max([dist1, dist2])

    if dist1 < dist2:
        idx_third_inside = idx_inside_polygon[int(len(idx_inside_polygon)/2)]
        if (maxx/2) < 250:
            ll = int(maxx/2)
        else:
            ll = 250
        idx_third_inside = idx_inside_polygon[np.argmin(abs(np.hypot(x_long[idx_inside_polygon] - x[middle_point_idx], y_long[idx_inside_polygon] - y[middle_point_idx]) - ll))]
        idx_third_inside2 = np.argmin(abs(np.hypot(x_long[:idx_inside_polygon[0]] - x[middle_point_idx], y_long[:idx_inside_polygon[0]] - y[middle_point_idx]) - 250))
        diff = idx_third_inside - idx_inside_polygon[0]
        diff2 = idx_third_inside2   #- idx_inside_polygon[0]
        x_transect, y_transect = x_long[diff2:idx_third_inside], y_long[diff2:idx_third_inside]
        transect_vector = np.column_stack((x_transect, y_transect))
        plt.plot(x_transect, y_transect, color='r')
        transect_inside_outside = transect_vector[[0, -1], :]
        
        if polygon_shoreline.contains(shapely.geometry.Point(transect_inside_outside[0, 0], transect_inside_outside[0, 1])):
            dict_transects[idx_transect] = transect_inside_outside
        else:
            dict_transects[idx_transect] = np.flip(transect_inside_outside, axis=0)

    else:
        
        idx_inside_polygon = np.flip(idx_inside_polygon)
        idx_third_inside = idx_inside_polygon[int(len(idx_inside_polygon)/2)]
        if (maxx/2) < 250:
            ll = int(maxx/2)
        else:
            ll = 250
        idx_third_inside = idx_inside_polygon[np.argmin(abs(np.hypot(x_long[idx_inside_polygon] - x[middle_point_idx], y_long[idx_inside_polygon] - y[middle_point_idx]) - ll))]
        idx_third_inside2 = np.argmin(abs(np.hypot(x_long[idx_inside_polygon[0]+1:] - x[middle_point_idx], y_long[idx_inside_polygon[0]+1:] - y[middle_point_idx]) - 250))
        diff = idx_inside_polygon[0] - idx_third_inside
        diff2 = idx_third_inside2 #idx_inside_polygon[0]  - 

        x_transect, y_transect = x_long[idx_third_inside:idx_inside_polygon[0]+1+idx_third_inside2], y_long[idx_third_inside:idx_inside_polygon[0]+1+idx_third_inside2]
        
        transect_vector = np.column_stack((x_transect, y_transect))
        plt.plot(x_transect, y_transect, color='b')
        transect_inside_outside = transect_vector[[0, -1], :]
        if polygon_shoreline.contains(shapely.geometry.Point(transect_inside_outside[0, 0], transect_inside_outside[0, 1])):
            dict_transects[idx_transect] = transect_inside_outside
        else:
            dict_transects[idx_transect] = np.flip(transect_inside_outside, axis=0)
    

    plt.axis('equal')


In [142]:
plt.plot(x_long[idx_inside_polygon[0]:], y_long[idx_inside_polygon[0]:])

In [151]:
plt.plot(idx_inside_polygon, np.hypot(x_long[idx_inside_polygon] - x[middle_point_idx], y_long[idx_inside_polygon] - y[middle_point_idx]) - 50)
plt.axhline(0)
np.argmin(abs(np.hypot(x_long[idx_inside_polygon] - x[middle_point_idx], y_long[idx_inside_polygon] - y[middle_point_idx]) - 50))

213

In [150]:
idx_inside_polygon[213]

2973